<a href="https://colab.research.google.com/github/pranaysingh97-bit/Harry-Potter-Agent/blob/main/Harrypotter_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the Google GenAI SDK and a powerful agent framework
!pip install -q google-genai crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.6/472.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.6/739.6 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 23.0 MB/

In [ ]:
import os
from google.colab import userdata

# Get your key from Google AI Studio and save it as a Colab Secret named 'GEMINI_API_KEY'
if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")

# The CrewAI LLM wrapper for Gemini
from crewai import LLM
from crewai_tools import SerperDevTool # Tool for the agent to look up information

# Initialize the Gemini model for your agent
# Gemini 2.5 Flash is fast and capable for this creative task
my_llm = LLM(
    model='gemini/gemini-2.5-flash',
    api_key=os.environ["GEMINI_API_KEY"],
    temperature=0.7 # Set a creative temperature
)

In [ ]:
persona_backstory = (
    "You are J.K. Rowling, the legendary author of the 'Harry Potter' series. "
    "You now write insightful, engaging LinkedIn posts to teach modern career "
    "and life lessons, always using a creative analogy from the Harry Potter books. "
    "Your tone is wise, inspiring, and slightly mystical, maintaining the spirit "
    "of an author speaking to her adult readers. You must always reference at "
    "least one key character (like Dumbledore, Snape, or Hermione) in your post."
)

In [ ]:
# Create a tool for real-time information lookup
# You'll need to set a SERPER_API_KEY in your Colab Secrets as well
search_tool = SerperDevTool(api_key=userdata.get("SERPER_API_KEY"))

# Note: The ReAct pattern is 'Thought -> Action (Tool) -> Observation -> Thought -> Final Answer'
# The agent will use this tool when it needs external knowledge.

In [ ]:
from crewai import Agent, Task, Crew, Process

# 🧙 The J.K. Rowling Agent
rowling_agent = Agent(
    role='LinkedIn Post Creator & Analogy Specialist',
    goal='Create a highly engaging, original LinkedIn post on a user-provided topic, '
         'grounded in the persona of J.K. Rowling, and using an insightful Harry Potter analogy.',
    backstory=persona_backstory,
    tools=[search_tool], # Gives the agent the ability to 'Act' (A) based on 'Reasoning' (R)
    llm=my_llm,
    verbose=True, # Set to True to see the internal ReAct steps (Thought, Action, Observation)
    # The CrewAI framework provides memory by default for the entire task context.
)

# 📝 The Specific Task
def create_linkedin_task(topic):
    return Task(
        description=(
            f"Write a single, creative LinkedIn post (4-5 paragraphs, maximum 200 words) "
            f"about the topic: '{topic}'. The post MUST: "
            f"1. Start with an engaging, hook-like opening sentence."
            f"2. Incorporate an analogy from the 'Harry Potter' universe. "
            f"3. Name at least one character (e.g., Albus Dumbledore, Severus Snape, Hagrid) to illustrate the lesson. "
            f"4. End with a question to drive engagement (e.g., 'What's your biggest life lesson from Hogwarts?'). "
            f"5. Include relevant LinkedIn hashtags (e.g., #Leadership #Career #HarryPotter #CreativeThinking)."
        ),
        agent=rowling_agent,
        expected_output="A final, polished LinkedIn post, ready to copy-paste.",
    )

In [ ]:
# --- User Input ---
topic = input("Enter the topic for J.K. Rowling's LinkedIn post (e.g., 'Overcoming self-doubt in a new career'): ")

# --- Setup the Crew ---
crew = Crew(
    agents=[rowling_agent],
    tasks=[create_linkedin_task(topic)],
    verbose=True, # Verbose level 2 shows agent and task execution details
    process=Process.sequential # Since it's a single agent, the process is sequential
)

# --- Execute the Agent ---
print("\n" + "="*50)
print(f"🪄 Generating J.K. Rowling's Post on: {topic}")
print("="*50)

# The result of this execution is your final, creative post
result = crew.kickoff()

print("\n" + "="*50)
print("✨ FINAL GENERATED LINKEDIN POST ✨")
print("="*50)
print(result)

Enter the topic for J.K. Rowling's LinkedIn post (e.g., 'Overcoming self-doubt in a new career'): Explain cash conversion cycle

🪄 Generating J.K. Rowling's Post on: Explain cash conversion cycle


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: fb6ae70a-c218-45aa-a35a-4c5edace1bc9                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: LinkedIn Post Creator & Analogy Specialist                                                              │
│                                                                                                                 │
│  Task: Write a single, creative LinkedIn post (4-5 paragraphs, maximum 200 words) about the topic: 'Explain     │
│  cash conversion cycle'. The post MUST: 1. Start with an engaging, hook-like opening sentence.2. Incorporate    │
│  an analogy from the 'Harry Potter' universe. 3. Name at least one character (e.g., Albus Dumbledore, Severus   │
│  Snape, Hagrid) to illustrate the lesson. 4. End with a question to drive engagement (e.g., 'What's your        │
│  biggest life lesson from Hogwarts?'). 5. Include relevant LinkedIn hashtags (e.g., #Leadership #Career         │
│  #HarryPotter #CreativeThinking).                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: LinkedIn Post Creator & Analogy Specialist                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Imagine Fred and George Weasley, always innovating, always looking for a Galleon. Their entrepreneurial        │
│  spirit, much like the flow of magic itself, depends on a vital rhythm: the Cash Conversion Cycle.              │
│                                                                                                                 │
│  It's the ultimate measure of how swiftly a business turns its investments – its "ingredients" – into hard      │
│  cash. Think of it: first, **Days Inventory Outstanding (DIO)**: how long those Skiving Snackbox components     │
│  sit in their workshop before becoming a finished, sellable product. The quicker they brew, the better!         │
│                                                                                                                 │
│  Next, **Days Sales Outstanding (DSO)**: once a student buys a Puking Pastille, how long until Fred and George  │
│  actually receive the Galleons? No credit at Hogwarts, thankfully, but in the wider world, this wait can be     │
│  significant. Finally, **Days Payable Outstanding (DPO)**: how long do they take to pay the gnomes who supply   │
│  their Fainting Fancies' unique ingredients? A shrewd delay can keep Galleons in their pocket longer.           │
│                                                                                                                 │
│  Albus Dumbledore, ever the master strategist, would surely appreciate the elegance of a well-managed cycle. A  │
│  shorter cycle means more Galleons available for new ventures, new inventions, new mischief! It's about         │
│  keeping your financial "magic" flowing without interruption.                                                   │
│                                                                                                                 │
│  What business lesson from the wizarding world resonates most with you?                                         │
│                                                                                                                 │
│  #CashConversionCycle #BusinessStrategy #HarryPotter #FinancialLiteracy #Entrepreneurship                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: fb6ae70a-c218-45aa-a35a-4c5edace1bc9                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Imagine Fred and George Weasley, always innovating, always looking for a Galleon. Their          │
│  entrepreneurial spirit, much like the flow of magic itself, depends on a vital rhythm: the Cash Conversion     │
│  Cycle.                                                                                                         │
│                                                                                                                 │
│  It's the ultimate measure of how swiftly a business turns its investments – its "ingredients" – into hard      │
│  cash. Think of it: first, **Days Inventory Outstanding (DIO)**: how long those Skiving Snackbox components     │
│  sit in their workshop before becoming a finished, sellable product. The quicker they brew, the better!         │
│                                                                                                                 │
│  Next, **Days Sales Outstanding (DSO)**: once a student buys a Puking Pastille, how long until Fred and George  │
│  actually receive the Galleons? No credit at Hogwarts, thankfully, but in the wider world, this wait can be     │
│  significant. Finally, **Days Payable Outstanding (DPO)**: how long do they take to pay the gnomes who supply   │
│  their Fainting Fancies' unique ingredients? A shrewd delay can keep Galleons in their pocket longer.           │
│                                                                                                                 │
│  Albus Dumbledore, ever the master strategist, would surely appreciate the elegance of a well-managed cycle. A  │
│  shorter cycle means more Galleons available for new ventures, new inventions, new mischief! It's about         │
│  keeping your financial "magic" flowing without interruption.                                                   │
│                                                                                                                 │
│  What business lesson from the wizarding world resonates most with you?                                         │
│                                                                                                                 │
│  #CashConversionCycle #BusinessStrategy #HarryPotter #FinancialLiteracy #Entrepreneurship                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): y


╭───────────────────────────────────────── 🔍 Execution Trace Generated ──────────────────────────────────────────╮
│                                                                                                                 │
│  🎉 Your First CrewAI Execution Trace is Ready!                                                                 │
│                                                                                                                 │
│  View your execution details here:                                                                              │
│  https://app.crewai.com/crewai_plus/ephemeral_trace_batches/15fa27ae-c4bb-4c1a-872b-165891b62253?access_code=T  │
│  RACE-3419ba464c                                                                                                │
│                                                                                                                 │
│  This trace shows:                                                                                              │
│  • Agent decisions and interactions                                                                             │
│  • Task execution timeline                                                                                      │
│  • Tool usage and results                                                                                       │
│  • LLM calls and responses                                                                                      │
│                                                                                                                 │
│  ✅ Tracing has been enabled for future runs! (CREWAI_TRACING_ENABLED=true added to .env)                       │
│  You can also add tracing=True to your Crew(tracing=True) / Flow(tracing=True) for more control.                │
│                                                                                                                 │
│  📝 Note: This link will expire in 24 hours.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


✨ FINAL GENERATED LINKEDIN POST ✨
Imagine Fred and George Weasley, always innovating, always looking for a Galleon. Their entrepreneurial spirit, much like the flow of magic itself, depends on a vital rhythm: the Cash Conversion Cycle.

It's the ultimate measure of how swiftly a business turns its investments – its "ingredients" – into hard cash. Think of it: first, **Days Inventory Outstanding (DIO)**: how long those Skiving Snackbox components sit in their workshop before becoming a finished, sellable product. The quicker they brew, the better!

Next, **Days Sales Outstanding (DSO)**: once a student buys a Puking Pastille, how long until Fred and George actually receive the Galleons? No credit at Hogwarts, thankfully, but in the wider world, this wait can be significant. Finally, **Days Payable Outstanding (DPO)**: how long do they take to pay the gnomes who supply their Fainting Fancies' unique ingredients? A shrewd delay can keep Galleons in their pocket longer.

Albus Dumbledore,